In [125]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import numpy as np
import nltk
import math
import os

In [162]:
# Initial question: can you predict crop yield 
# by year based on the total exploitable water resources a country has available?


def convert_to_pandas(filename):
    """
    Convert dataset stored in data/ dir with a filename descriptor into a pandas df
    
    filename - name of file in data/{filename}: string
    """
    if "xlsx" in filename:
        return pd.read_excel(f"data/{filename}")
    if "csv" in filename:
        return pd.read_csv(f"data/{filename}")
    
def crop_yield_specialized_preprocessing(df, total_yield_output=True):
    """
    Performs crop yield specialized preprocessing
    Our class column dataset requires special preprocessing to convert the attainable yields and yield gaps
    into crop yield
    
    df - crop yield input dataframe: DataFrame
    """
    attainable_cols = [col for col in df.columns if "attainable" in col]
    gap_cols = [col for col in df.columns if "gap" in col]
    crop_names = [name.split("_")[0] for name in attainable_cols]
    new_col_names = list()
    
    for crop in crop_names:
        attainable_col_name = [col for col in attainable_cols if crop in col][0]
        gap_col_name = [col for col in gap_cols if crop in col][0]

        new_col_name = f"{crop}_crop_yield"
        new_col_names.append(new_col_name)
        df[new_col_name] = df[attainable_col_name] - df[gap_col_name]
    
    df.rename(columns = {'Entity':'Country'}, inplace = True)
    
    if total_yield_output:
        df['total_yield'] = df[new_col_names].sum(axis=1)
        df = strip_df(df, ["Country", "Year", "total_yield"]) 
        return df
            
    df = strip_df(df, ["Country", "Year"] + new_col_names)
    return df


def aquastat_feature_rename(df, fn, aquastat_feature):
    new_column_value = fn.split(".xlsx")[0]
    df.rename(columns = {aquastat_feature: new_column_value}, inplace = True)
    return df
    
    
    
def strip_df(df, keep_features_list):
    """
    Strips a dataframe of anything except whats included in the keep_features_list
    
    df - input df: DataFrame
    keep_features_list - list of features you want to keep as column name : list[str]
    """
    del_columns = [column for column in df.columns if column not in keep_features_list]
    for col in del_columns:
        df = df.drop([col], axis=1)
    return df
    
def combine_feature_dfs(dfs, feature_merge_list):
    """
    Merges two dataframes based on the column name values provided in the feature_merge_list
    
    df1 - list of input dfs: List[DataFrame]
    feature_merge_list - list of features you want to merge by (Year and Country usually) : list[str]
    """
    
    aquastat_df = dfs[0]
    for aqua_df in dfs[1:]:
        aquastat_df = aquastat_df.merge(aqua_df, on=feature_merge_list, how='outer')
    
    return aquastat_df


def pipeline(aquastat_filenames, crop_yield_filename, aquastat_feature):
    """
    Runs the aquastat and crop yield pipeline
    """
    shared_attributes = ["Country", "Year"]
    crop_df = crop_yield_specialized_preprocessing(convert_to_pandas(crop_yield_filename))
    dfs = []
    for fn in aquastat_filenames:
        temp_df = convert_to_pandas(fn)
        strip_temp_df = strip_df(temp_df, shared_attributes + [aquastat_feature])
        renamed_temp_df = aquastat_feature_rename(strip_temp_df, fn, aquastat_feature)
        renamed_temp_df = renamed_temp_df.dropna()
        dfs.append(renamed_temp_df)
    dfs.append(crop_df)
    combined_df = combine_feature_dfs(dfs, shared_attributes) 
    
    return combined_df.dropna()


In [163]:
# Combining total exploitable water resources (10^9 m3/year) 
# with the various crop yeilds by country and by year

aquastat_file_names = [fn for fn in os.listdir("data/") if ".xlsx" in fn]

aquastat_crop_yeild = pipeline(aquastat_file_names, "Attainable_yields.csv", "value")

aquastat_crop_yeild

,Country,Year,total_renewable_water_resources,agricultural_value_added_percent_of_gdp,total_exploitable_water_resources,precipitation_index,gdp_per_capita,dam_capacity,total_population_with_access_to_safe_drinking_water,total_internal_renewable_water_resource_per_capita,arable_land_area,total_population,total_yield
59,Albania,2018,30.2,18.429476,13.0,1136.000000,5257.650696,4.030000,95.1,9331.399988,611.346,2882.740,87.14
60,Albania,2017,30.2,19.022127,13.0,1136.000000,4514.204908,4.030000,95.1,9326.776621,612.000,2884.169,69.81
61,Albania,2016,30.2,19.849993,13.0,1136.000000,4109.340457,4.030000,95.1,9319.444935,620.300,2886.438,68.46
62,Albania,2015,30.2,19.780225,13.0,1136.000000,3939.413126,4.030000,95.1,9306.306528,615.100,2890.513,71.04
63,Albania,2014,30.2,19.990153,13.0,1136.000000,4567.281443,4.030000,95.1,9287.695875,615.600,2896.305,68.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10412,Zimbabwe,1994,20.0,13.948258,1.5,732.125000,946.224014,99.889946,79.5,1088.641333,3100.000,11261.744,142.84
10413,Zimbabwe,1993,20.0,11.218912,1.5,733.500000,913.671452,99.489946,79.6,1105.224792,3030.000,11092.766,95.91
10414,Zimbabwe,1992,20.0,5.513596,1.5,666.566667,957.173183,99.489946,79.6,1124.718843,2990.000,10900.502,125.17
10415,Zimbabwe,1991,20.0,11.062933,1.5,599.633333,1184.578132,99.489946,79.5,1147.833137,2950.000,10680.995,138.35
